In [ ]:
# ===============================================
# Trénink modelu sentimentové analýzy (čeština)
# Autor: Radka Fojtová
# Popis: Tento notebook obsahuje skript pro trénink
# předtrénovaného modelu RobeCzech pro klasifikaci
# vět podle sentimentu (pozitivní, neutrální, negativní).
# ===============================================

In [ ]:
# Připojení na drive
from google.colab import drive
drive.mount('/content/drive')
import os


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Soubory ve složce csfd: ['neutral.txt', 'licence.txt', 'negative.txt', 'positive.txt']


In [ ]:
# Načtení příspěvků a štítků z Facebooku
# Změňte na cestu ke složce s anotovanými facebookovými příspěvky
fb_path = "vase_cesta_k_datům"
with open(f"{fb_path}/gold-posts.txt", encoding="utf-8") as f:
    posts = [line.strip() for line in f.readlines()]

with open(f"{fb_path}/gold-labels.txt", encoding="utf-8") as f:
    labels = [line.strip() for line in f.readlines()]

filtered = [(text, label) for text, label in zip(posts, labels) if label in ["p", "n", "0"]]

label_map = {"p": 2, "0": 1, "n": 0}
filtered = [{"text": text, "label": label_map[label]} for text, label in filtered]

import pandas as pd
df_fb = pd.DataFrame(filtered)

print(f"Facebook příspěvků po vyfiltrování: {len(df_fb)}")
df_fb.sample(5)


Facebook příspěvků po vyfiltrování: 9752


,text,label
8826,já chci Mc Bůůůček!!! :-),1
3538,Ta omacka nechutná :! :D,0
1011,"Tak další dobrý úlet , reklamky tohoto typu do...",1
2850,Co je levnější než zadarmo...? :-) :-) :-),1
8175,kiki já mám taky chytrý telefon a žadné proble...,2


In [ ]:
import os
import pandas as pd

def load_dataset_from_folder(folder_path):
    data = []
    files = {
        "negative.txt": 0,
        "neutral.txt": 1,
        "positive.txt": 2
    }
    for file_name, label in files.items():
        file_path = os.path.join(folder_path, file_name)
        with open(file_path, "r", encoding="utf-8") as f:
            lines = f.readlines()
            data.extend([(line.strip(), label) for line in lines if line.strip()])
    return pd.DataFrame(data, columns=["text", "label"])

# Načtení čsfd a mall.cz
# Změňte na cestu ke složce s csfd a mallcz daty
base_path = "vase_cesta_k_datům"
df_csfd = load_dataset_from_folder(os.path.join(base_path, "csfd"))
df_mallcz = load_dataset_from_folder(os.path.join(base_path, "mallcz"))


# Načtení jemně negativních vět
# Změňte na cestu ke složce s vašimi větami, pokud máte
df_softneg = pd.read_csv("vase_cesta_k_datům")

# Spojení do jednoho datasetu
df_all = pd.concat([df_csfd, df_mallcz, df_fb, df_softneg], ignore_index=True)
df_all = df_all.sample(frac=1).reset_index(drop=True)

print(f"CELKEM vět ve všech datasetech: {len(df_all)}")
df_all.sample(10)



CELKEM vět ve všech datasetech: 258440


,text,label
122905,Super větší balení. Jako originál Kinley...,2
143496,"Spokojena - doporučuji, šněrování do špičky um...",1
97018,Maximální spokojenost. Zápach z potu téměř nee...,2
182395,"spokojenost, doporucuji :o)",2
129018,"Vyhovující, jen je potřeba umístit externí čid...",0
164158,Nejlepší seriál tehdejší doby díky dobrému obs...,2
87980,Vzhledem k ceně velmi dobrý výrobek. Jednoduch...,1
33510,Zatím jsem nenašla nic co by nebylo podle mých...,2
199619,"Jsem sice začátečník, ale výrobek mi poskytuje...",2
108659,"Zástupce té slabší skupiny mayovek, kde to nav...",1


In [ ]:
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [ ]:
from transformers import AutoTokenizer

# Načtení tokenizeru z RobeCzech
tokenizer = AutoTokenizer.from_pretrained("ufal/robeczech-base")

# Funkce pro tokenizaci
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

from datasets import Dataset
from sklearn.model_selection import train_test_split

# Rozdělení na trénovací a validační data
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df_all["text"].tolist(), df_all["label"].tolist(), test_size=0.1
)

train_dataset = Dataset.from_dict({"text": train_texts, "label": train_labels})
val_dataset = Dataset.from_dict({"text": val_texts, "label": val_labels})

# Tokenizace
train_dataset = train_dataset.map(tokenize, batched=True)
val_dataset = val_dataset.map(tokenize, batched=True)

# Nastavení formátu pro PyTorch
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
val_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/209 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/516 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/955k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/601k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.66M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/963 [00:00<?, ?B/s]

Map:   0%|          | 0/232596 [00:00<?, ? examples/s]

Map:   0%|          | 0/25844 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd
import numpy as np
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Nastavení modelu a trénovacích parametrů – počet epoch, batch size, learning rate, logování atd.


model = RobertaForSequenceClassification.from_pretrained("ufal/robeczech-base", num_labels=3)

training_args = TrainingArguments(
    output_dir="./results-robeczech",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    learning_rate=2e-5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(labels, predictions),
        "f1_macro": f1_score(labels, predictions, average="macro")
    }

# Spuštění tréninku

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()


model.safetensors:   0%|          | 0.00/504M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ufal/robeczech-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-8-27fe813ac823>:39: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: fojtovaradka2 (fojtovaradka2-masarykova-univerzita) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro
1,0.435500,0.425626,0.832147,0.800619
2,0.350900,0.396775,0.848630,0.827016
3,0.283800,0.427416,0.856833,0.835108


TrainOutput(global_step=43614, training_loss=0.38068534747690624, metrics={'train_runtime': 28952.649, 'train_samples_per_second': 24.101, 'train_steps_per_second': 1.506, 'total_flos': 1.8359738552958566e+17, 'train_loss': 0.38068534747690624, 'epoch': 3.0})